### IMPORT DE BIBLIOTECAS

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os

import implicit
from implicit.evaluation import (train_test_split, 
                                 ndcg_at_k, 
                                 AUC_at_k,
                                 mean_average_precision_at_k, 
                                 precision_at_k)

import recometrics
from cmfrec import MostPopular
from lightfm import LightFM

from scipy.sparse import (csr_matrix, 
                          save_npz, 
                          load_npz)
import scipy

from matplotlib import pyplot as plt
import seaborn as sns

import pickle
from tqdm import tqdm
import warnings


In [2]:
warnings.filterwarnings('ignore')

### Carregamento de dados de categorias e histórico de clicks

In [3]:
df_it_cat = pd.read_csv(os.getcwd()+'/.csv/new_category_sample.csv') # DataFrame Informativo das categorias

In [4]:
clicks_it = pd.read_csv(os.getcwd()+'/.txt/clicks_it_sample.txt', sep = ',', header=0) # df de histórico de clicks

Valores missing na coluna de OfferTitle que serão excluídos.

In [5]:
clicks_it = clicks_it[clicks_it.OfferTitle.isna() == False]

### Conversão das colunas de usuário e oferta em categórica e criando novas colunas com os códigos adotados 

In [6]:
clicks_it.UserId = clicks_it.UserId.astype('category')
clicks_it.OfferId = clicks_it.OfferId.astype('category')

In [7]:
clicks_it['User'] = clicks_it.UserId.cat.codes
clicks_it['Offer'] = clicks_it.OfferId.cat.codes

### CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA

In [8]:
clicks_it.drop(columns = ['Keywords'], axis = 1, inplace=True)  # DROP DAS KEYWORDS

#df_it_cat.rename({'Ancertor_ID':'Ancestor_ID'}, axis = 1, inplace = True) # RENAME DO ANCESTOR_ID

#df_it_cat.drop('Unnamed: 0', axis =1, inplace = True) # Remoção de coluna Unnamed

clicks_it['Cat_clicks'] = clicks_it.groupby('Category')['OfferId'].transform('count') # Criação de coluna de clicks por categoria

In [9]:
df_it_cat = df_it_cat[df_it_cat.Country == 'fr'].drop('Country',axis =1)

### Merge do Dataframe de categorias com o dataframe de clicks 

In [10]:
clicks_it = clicks_it.merge(df_it_cat, on = 'Category')

### Criação de coluna com o nº total de clicks do usuário e nº total de clicks do produto

In [11]:
clicks_it['UserTotalClicks'] = clicks_it.groupby(by=['UserId'])['OfferId'].transform('count')

In [12]:
clicks_it['ProductClicks'] = clicks_it.groupby(by='OfferId')['UserId'].transform('count')

In [19]:
def set_params():
    params = []
    for min_product in [20,25, 30, 40, 45, 50]:
        for max_product in [0, 1, 2]:
            for min_users in [5, 7, 9, 15]:
                clicks_it_filtered = clicks_it[(clicks_it.ProductClicks > min_product) & (clicks_it.ProductClicks < (clicks_it.ProductClicks.mean() + max_product*clicks_it.ProductClicks.std()))]

                clicks_it_filtered['UserTotalClicks'] = clicks_it_filtered.groupby(by=['UserId'])['OfferId'].transform('count')
                clicks_it_filtered = clicks_it_filtered[((clicks_it_filtered.UserTotalClicks > min_users))]

                clicks_it_filtered.UserId = clicks_it_filtered.UserId.astype('category')
                clicks_it_filtered.OfferId = clicks_it_filtered.OfferId.astype('category')


                clicks_it_filtered['User'] = clicks_it_filtered.UserId.cat.codes
                clicks_it_filtered['Offer'] = clicks_it_filtered.OfferId.cat.codes

                clicks_per_user_product = clicks_it_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)
                sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], 
                                               (clicks_per_user_product['User'], 
                                                clicks_per_user_product['Offer'])))

                sparse_user_item = (sparse_user_item).astype('double')
                kfold = []
                for i in range(3):
                    train, test, model = als_model(sparse_user_item)
                    kfold.append(implicit.evaluation.precision_at_k(model, train, test))
                    precision = np.array(kfold).mean()
                params.append([min_product, max_product, min_users, precision])
                
    full_params = pd.DataFrame(params, columns = ['min_product','max_product', 'min_users', 'precision'])
    return full_params

In [20]:
full = set_params()

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/861 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/879 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/97 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

In [45]:
full.sort_values(by = 'precision', ascending = False)[:10]

,min_product,max_product,min_users,precision
60,50,0,5,0.250192
61,50,0,7,0.223626
48,45,0,5,0.221666
50,45,0,9,0.218096
36,40,0,5,0.214279
68,50,2,5,0.204849
56,45,2,5,0.195974
64,50,1,5,0.193549
37,40,0,7,0.193372
25,30,0,7,0.192858


In [116]:
#clicks_it_filtered.UserId = clicks_it_filtered.UserId.astype('category')
# clicks_it_filtered.OfferId = clicks_it_filtered.OfferId.astype('category')


# clicks_it_filtered['User'] = clicks_it_filtered.UserId.cat.codes
# clicks_it_filtered['Offer'] = clicks_it_filtered.OfferId.cat.codes#Cap minimo de clicks para integrar o sistema de recomendação
clicks_it_filtered = clicks_it[(clicks_it.ProductClicks > 40) & (clicks_it.ProductClicks < (clicks_it.ProductClicks.mean() + 2*clicks_it.ProductClicks.std()))]

In [117]:
clicks_it_filtered['UserTotalClicks'] = clicks_it_filtered.groupby(by=['UserId'])['OfferId'].transform('count')
clicks_it_filtered = clicks_it_filtered[(clicks_it_filtered.UserTotalClicks > 7)]

In [118]:
clicks_it_filtered.to_csv('it_model_sample.csv')


### Agrupamento de dos clicks de usuário em ofertas únicas para termos a quantidade de cada usuário em cada oferta.


In [119]:
clicks_per_user_product = clicks_it_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)

In [120]:
len(clicks_per_user_product.User.unique()), len(clicks_per_user_product.Offer.unique())

(1048, 1500)

### Criação de matrizes esparsas Usuário-item e item-usuário

In [121]:
alpha = 40
sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['User'], clicks_per_user_product['Offer'])))

sparse_user_item = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente
data = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente

In [122]:
#Esparsidade de matriz
possible_interactions = sparse_user_item.shape[0]*sparse_user_item.shape[1]
interacted = len(sparse_user_item.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsity

0.9999999480033447

 Mais de 99.99% das interações possíveis entre usuários e produtos na atual base dados não foi ainda realizada. Segundo artigo: For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. Devemos reavaliar a matriz?

In [96]:
save_npz(os.getcwd()+"/.npz/fr/sparse_user_item.npz", sparse_user_item)

In [97]:
model_path = os.getcwd()+'/.pkl/it/it_als_model.pkl'

* Criação de diferentes matrizes esparsas para operar com o algoritmo. Usuário-item e item-usuário. Cada uma deve ser usada no momento preciso
* O alfa é o coeficiente de confiabilidade da interação do usuário com um item específico. Valor utilizado fi adotado com base no artigo: https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b. Mas, podemos testar outros valores na validação do modelo.
* Outro artigo de base pra elaboração do modelo: https://medium.com/analytics-vidhya/implementation-of-a-movies-recommender-from-implicit-feedback-6a810de173ac

# FUNÇÃO DE RECOMENDAÇÕES - IMPLICIT

## Treinamento de modelos

In [98]:
offers = pickle.load(open(os.getcwd()+"/.pkl/fr/offers.pkl", "rb"))

Carregamentodo dicionário que converte os códigos de ofertas para o seu título de oferta. Ainda falta traduzir do alemão para o inglês para tirar mais significado dos resultados

In [123]:
def als_model(sparse_user_item):
    
    '''computes p@k and map@k evaluation metrics and saves model'''
    
    #sparse_item_user = load_npz(os.getcwd()+"/.npz/sparse_item_user.npz")
      
    train, test = implicit.evaluation.train_test_split(sparse_user_item, train_percentage=0.8)

    model = implicit.als.AlternatingLeastSquares(factors=50, 
                                                 regularization=0.1, 
                                                 iterations=50,
                                                 calculate_training_loss=False)
    alpha=40
    model.fit(train*alpha)

    with open(model_path, 'wb') as pickle_out:
        pickle.dump(model, pickle_out)
    
    return train, test, model

In [124]:
train, test, model = als_model(sparse_user_item)
implicit.evaluation.precision_at_k(model,train,test)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/869 [00:00<?, ?it/s]

0.2120470127326151

### Avaliação do modelo

In [125]:
def model_evaluation(train, test, model, k=10): 
    
    '''Avaliação do modelo treinado com as funções da biblioteca Implicit.
    Retorna dicionário com p@k, map@k, ndcg@k e auc@k.'''

    
    p_at_k = implicit.evaluation.precision_at_k(model, train_user_items=train, 
                                                test_user_items=test,
                                                K=k, 
                                                show_progress = False)
    
    m_at_k = implicit.evaluation.mean_average_precision_at_k(model, 
                                                             train_user_items = train, 
                                                             test_user_items = test, 
                                                             K=k, 
                                                             show_progress = False)

    ndcg_at_k = implicit.evaluation.ndcg_at_k(model, 
                                              train_user_items = train,
                                              test_user_items = test, 
                                              K=k, 
                                              show_progress = False)

    auc_at_k = implicit.evaluation.AUC_at_k(model, 
                                            train_user_items = train, 
                                            test_user_items = test, 
                                            K=k, 
                                            show_progress = False)
    metrics = {'p@K':p_at_k, 
               'map@K': m_at_k, 
               'ndcg@K':ndcg_at_k, 
               'auc@K':auc_at_k}
    
    return metrics

In [126]:
metrics = model_evaluation(train, test, model, k=10)
metrics

{'p@K': 0.2120470127326151,
 'map@K': 0.12213160408006703,
 'ndcg@K': 0.16711622039723031,
 'auc@K': 0.6207093306006258}

## Avaliação Recometrics

In [104]:
X_train, X_test, users_test = \
    recometrics.split_reco_train_test(
        sparse_user_item, split_type="joined",
        users_test_fraction = None,
        max_test_users = 10000,
        items_test_fraction = 0.3
    )

In [105]:
### Random recommendations (random latent factors)
rng = np.random.default_rng(seed=1)
UserFactors_random = rng.standard_normal(size=(X_test.shape[0], 5))
ItemFactors_random = rng.standard_normal(size=(X_test.shape[1], 5))

### Non-personalized recommendations
model_baseline = MostPopular(implicit=True, user_bias=False).fit(X_train.tocoo())
item_biases = model_baseline.item_bias_
item_biases

array([0., 0., 0., ..., 0., 0., 0.])

In [127]:
### Fitting WRMF model
wrmf = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations = 50, random_state=123)
wrmf.fit(X_train*40)

### Fitting BPR model with WARP loss
bpr_warp = LightFM(no_components=50, loss="warp", random_state=123)
bpr_warp.fit((X_train*40).tocoo())

  0%|          | 0/50 [00:00<?, ?it/s]

In [141]:
k = 5 ## Top-K recommendations to evaluate

metrics_random = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    UserFactors_random, ItemFactors_random,
    k=k, all_metrics=True
)

metrics_baseline = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    None, None, item_biases=item_biases,
    k=k, all_metrics=True
)

metrics_wrmf = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    wrmf.user_factors[:X_test.shape[0]], wrmf.item_factors,
    k=k, all_metrics=True
)

metrics_bpr_warp = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    bpr_warp.user_embeddings[:X_test.shape[0]], bpr_warp.item_embeddings,
    item_biases=bpr_warp.item_biases,
    k=k, all_metrics=True
)

In [142]:
all_metrics = [
    metrics_random,
    metrics_baseline,
    metrics_wrmf,
    metrics_bpr_warp
]
all_metrics = pd.concat([m.mean(axis=0).to_frame().T for m in all_metrics], axis=0)
all_metrics.index = [
    "Random",
    "Non-personalized",
    "WRMF (a.k.a. iALS)",
    "BPR-WARP"
]
all_metrics

,P@5,TP@5,R@5,AP@5,TAP@5,NDCG@5,Hit@5,RR@5,ROC_AUC,PR_AUC
Random,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.492573,0.000077
Non-personalized,0.011807,0.021747,0.021386,0.013089,0.013353,0.021096,0.055422,0.036345,0.962783,0.030154
WRMF (a.k.a. iALS),0.077350,0.159679,0.157906,0.100958,0.101993,0.136054,0.291566,0.180843,0.769626,0.142418
BPR-WARP,0.020000,0.032992,0.032068,0.017762,0.018382,0.029473,0.084337,0.046827,0.955534,0.038958


In [143]:
all_metrics = all_metrics.apply(lambda x: round(x,3))
all_metrics

,P@5,TP@5,R@5,AP@5,TAP@5,NDCG@5,Hit@5,RR@5,ROC_AUC,PR_AUC
Random,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.493,0.000
Non-personalized,0.012,0.022,0.021,0.013,0.013,0.021,0.055,0.036,0.963,0.030
WRMF (a.k.a. iALS),0.077,0.160,0.158,0.101,0.102,0.136,0.292,0.181,0.770,0.142
BPR-WARP,0.020,0.033,0.032,0.018,0.018,0.029,0.084,0.047,0.956,0.039


In [144]:
all_metrics.to_csv('.csv/it/it_all_metrics_5.csv')

Fonte: https://nb.recohut.com/implicit/music/evaluation/2021/07/13/evaluation-recometrics-lightfm.html

* Sobre metricas de precisão @k: https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
* Sobre NDCG: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
* Sobre Mean Average Precision: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52

Ainda não foi realizada qualquer tunagem de hiperaparâmetros. Podemos pegar alguns valores de referencia para rodar um gridsearch

## Funções de recomendações

In [143]:
def recommend(user, K=10):
    
    ''' Retorna uma lista de itens recomendados para o usuário dado de acordo com a biblioteca Implicit.
        Também é retornado uma lista com os itens já clicados por esse usuário'''
    
    #sparse_user_item = load_npz("/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
    
    if user not in sparse_user_item.T.tocsr().indices:
        return "Invalid User"
        
    recommended, scores = (model.recommend(user, sparse_user_item[user], K))

    original_user_items = list(sparse_user_item[user].indices)

    return recommended, original_user_items

* A matriz m passa a ser a matriz com o novo usuário atualizado e é levada em consideração no para o cálculo de novos vetores.

Nota: 
* Após os ajustes na organização das matrizes esparsas, o modelo parece não mais repetir recomendações de itens que já foram clicados pelo usuário
* O modelo parece também não mais necessitar de tradução dos códigos de ofertas e usuário adotados na matriz esparsa para os códigos da matriz original

### Criação e armazenamento do dicionário código-titulo de oferta (carregado na parte de cima do código).

In [22]:
df_temp = clicks_it[['Offer','OfferTitle']]

In [23]:
df_temp = df_temp.drop_duplicates(['Offer','OfferTitle'])

In [24]:
offers = dict(zip(df_temp['Offer'], df_temp['OfferTitle']))

In [25]:
#dic = pd.read_csv('de_traduzido.csv')
#dic['Offer2'] = dic['OfferTitle'].map(dict(zip(clicks_it.OfferTitle, clicks_it.Offer)))

In [26]:
pickle.dump(offers, open('.pkl/it/offers.pkl', 'wb'))

### Criação e armazenamento do dicionário código_oferta -> Filha_1 e código_oferta -> Categoria (carregado na parte de cima do código).

In [27]:
clicks_it.filha_1 = clicks_it.filha_1.astype('int32')

In [28]:
df_temp = clicks_it[['Offer','filha_1', 'Category']]

In [29]:
df_temp = df_temp.drop_duplicates(['Offer','filha_1', 'Category'])

In [30]:
offer_filha1 = dict(zip(df_temp['Offer'], df_temp['filha_1']))
pickle.dump(offer_filha1, open('.pkl/it/offer_sub1.pkl', 'wb'))

In [31]:
offer_cat = dict(zip(df_temp['Offer'], df_temp['Category']))
pickle.dump(offer_cat, open('.pkl/it/offer_cat.pkl', 'wb'))

### Tabela de info dos produtos

In [32]:
products_info = clicks_it[['Offer','Category','filha_1', 'ProductClicks']]

In [33]:
products_info.filha_1 = products_info.filha_1.astype('int32')

In [34]:
products_info = products_info.drop_duplicates().reset_index().drop('index', axis = 1)

In [35]:
products_info.rename({'filha_1':'SuperCat'}, axis = 1, inplace = True)

In [36]:
products_info.sort_values(by=['SuperCat','ProductClicks'], ascending = False, inplace = True)

In [37]:
products_info.to_csv('.csv/it/products_info.csv')

In [209]:
products_info.head()

,Offer,Category,SuperCat,ProductClicks
119765,197556,100513123,100450123,406
305519,44911,100508723,100450123,391
127018,45255,100334923,100450123,373
126916,70579,100334923,100450123,333
238397,225605,100450223,100450123,284


### Dicionário Nome Sub1

In [38]:
df = clicks_it[['filha_1','filha_1_name']].drop_duplicates()
sub1_name = dict(zip(df.filha_1, df.filha_1_name))
pickle.dump(sub1_name, open('.pkl/it/sub1_name.pkl', 'wb'))

### Dicionário Nome Cat

In [40]:
df = clicks_it[['Category','Translate']].drop_duplicates()
cat_name = dict(zip(df.Category, df.Translate))
pickle.dump(cat_name, open('.pkl/it/cat_name.pkl', 'wb'))


## Tunagem de hiperparametros

In [109]:
grid = {'factors': [10,50,75,100],
       'regularization':[0.1, 0.01],
        'iterations':[50, 75, 100], 
        'alphas':[40,60,80]}

In [110]:
model = implicit.als.AlternatingLeastSquares()

In [111]:
def gridsearch_als(grid):
    results = []
    for factor in grid['factors']:
        for regularization in grid['regularization']:
            for iteration in grid['iterations']:
                for alpha in grid['alphas']:
            
                    model = implicit.als.AlternatingLeastSquares(factors = factor, 
                                                                 regularization = regularization, 
                                                                 iterations = iteration)
                    model.fit(train*alpha)

                    partial = model_evaluation(train, test, model, )
                    results.append([factor, regularization, iteration, alpha, partial['p@K'],partial['map@K'], partial['ndcg@K'], partial['auc@K']])
                
    final = pd.DataFrame(results, columns = ['Factors','Regularization','Iteration','alpha','P@K','MAP@K','NDCG@K','AUC@K'])
    return final
                

In [112]:
grid_results = gridsearch_als(grid)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
grid_results.to_csv('params.csv')

In [113]:
grid_results.sort_values(by=['P@K','NDCG@K'], ascending = False)

,Factors,Regularization,Iteration,alpha,P@K,MAP@K,NDCG@K,AUC@K
18,50,0.10,50,40,0.233750,0.111450,0.162824,0.621524
24,50,0.10,100,40,0.233125,0.108114,0.159692,0.620603
33,50,0.01,100,40,0.231875,0.109272,0.160710,0.620285
21,50,0.10,75,40,0.231250,0.110869,0.161830,0.620718
30,50,0.01,75,40,0.231250,0.109785,0.161345,0.621795
...,...,...,...,...,...,...,...,...
17,10,0.01,100,80,0.187500,0.080332,0.122784,0.593278
3,10,0.10,75,40,0.186250,0.085649,0.129270,0.596751
0,10,0.10,50,40,0.185625,0.082625,0.126416,0.594794
12,10,0.01,75,40,0.184375,0.086151,0.128702,0.594177
